In [3]:
from transformers import BertModel, BertTokenizer, BertConfig, BertForMaskedLM
import torch
import random
import logging

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
random.seed(0)

In [5]:
# Load pre-trained model tokenizer (vocabulary) and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [33]:
def tokenize_input(text):
    tokenized_text = tokenizer.tokenize(text)
    
    sentence1 = tokenized_text[:tokenized_text.index(".")+1]
    sentence2 = tokenized_text[tokenized_text.index(".")+1:]
    
    sentence1 = ["[CLS]"] + sentence1
    sentence2 = ["[SEP]"] + sentence2 + ["[SEP]"]
    
    segments_ids = [0]*len(sentence1) + [1]*len(sentence2)
    new_tokenized_text = sentence1 + sentence2
    
    #print(len(sentence1))
    
    return new_tokenized_text, segments_ids

In [58]:
def mask_tokens(tokenized_text, mask_token_ids):
    for id in mask_token_ids:
        #print(tokenized_text[id])
        tokenized_text[id] = '[MASK]'
                
    # Convert tokenized text to their index
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    
    return tokenized_text, indexed_tokens

In [59]:
def produce_new_sentence(masked_tokenize, mask_token_ids, predictions):
    predicted = []

    for i in mask_token_ids:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

        predicted.append(predicted_token)
    
    print(mask_token_ids)
    print(predicted)

    for word in predicted:
        mask_index = masked_tokenize.index("[MASK]")
        masked_tokenize[mask_index] = word

    final = ' '.join(masked_tokenize).replace(" ##", '').replace("[CLS] ", "").replace("[SEP] ", "").replace("[SEP]", "")

    print(final)
    
    return final

In [61]:
text = """My countrymen and women of the radio audience Of the untold values of the radio, one is the great intimacy it has brought among our people. 
The greatest strength of the radio is the intimacy of its listeners."""

cycles = 40

#number_masked = int((len(tokenized_text) - period_loc+1) / 5)
number_masked = 1

tokenized_text, segments_ids = tokenize_input(text)

for c in cycles:
    period_loc = tokenized_text.index(".")
    
    mask_token_ids = sorted(random.sample(range(period_loc+2, len(tokenized_text)-1), number_masked))

    masked_tokenize, indexed_tokens = mask_tokens(tokenized_text, mask_token_ids)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]

    tokenized_text = produce_new_sentence(masked_tokenize, mask_token_ids, predictions)
    
    

the
[32]
['the']
my countrymen and women of the radio audience of the untold values of the radio , one is the great intimacy it has brought among our people . the greatest strength of the radio is the intimacy of its listeners . 
the
[39]
['the']
my countrymen and women of the radio audience of the untold values of the radio , one is the great intimacy it has brought among our people . the greatest strength of the radio is the intimacy of its listeners . 
listeners
[43]
['listeners']
my countrymen and women of the radio audience of the untold values of the radio , one is the great intimacy it has brought among our people . the greatest strength of the radio is the intimacy of its listeners . 
greatest
[33]
['great']
my countrymen and women of the radio audience of the untold values of the radio , one is the great intimacy it has brought among our people . the great strength of the radio is the intimacy of its listeners . 
radio
[37]
['radio']
my countrymen and women of the radio audien

In [106]:
# Round 2: Potentially Growing

text = """All of which comprises a very substantial reserve in the country at the present time . The Reserve Bank of India has also made a large amount of deposits , which is a good thing ."""

def tokenize_input2(text):
    tokenized_text = tokenizer.tokenize(text)
    
    sentence1 = tokenized_text[:tokenized_text.index(".")+1]
    sentence2 = tokenized_text[tokenized_text.index(".")+1:]
    
    sentence1 = ["[CLS]"] + sentence1
    sentence2 = ["[SEP]"] + sentence2 + ["[SEP]"]
    
    new_tokenized_text = sentence1 + sentence2
        
    return new_tokenized_text, sentence1, sentence2


tokenized_text, sentence1, sentence2 = tokenize_input2(text)
period_loc = tokenized_text.index(".")
segments_ids = [0]*len(sentence1) + [1]*len(sentence2)


def mask_tokens(tokenized_text, mask_token_ids):
    for id in mask_token_ids:
        print(f"{id} ie. {tokenized_text[id]} is masked")
        tokenized_text[id] = '[MASK]'
    
    #Convert tokenized text to their index
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    
    return tokenized_text, indexed_tokens



def predict_token(masked_tokenize, mask_token_ids, predictions):
    predicted = []

    for i in mask_token_ids:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

        predicted.append(predicted_token)

    for word in predicted:
        mask_index = masked_tokenize.index("[MASK]")
        
        print(f"Masked index: {mask_index} became {word}")
        masked_tokenize[mask_index] = word
        
        if word == "." and masked_tokenize[mask_index + 1] != "[SEP]":
            masked_tokenize.insert(mask_index + 1,"[SEP]")          
    
    return masked_tokenize

cycles = 1000
number_masked = 2
masked_tokenize_prior_length = 99999


for c in range(cycles):
    mask_token_ids = sorted(random.sample(range(period_loc+2, len(tokenized_text)-1), number_masked))
    
    masked_tokenize, indexed_tokens = mask_tokens(tokenized_text, mask_token_ids)
    
    if len(masked_tokenize) > masked_tokenize_prior_length:
        segments_ids = segment_ids.append(1)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]

    tokenized_text = predict_token(masked_tokenize, mask_token_ids, predictions)
    masked_tokenize_prior_length = len(tokenized_text)
    
    print(tokenized_text)
    print("==============================")

28 ie. amount is masked
30 ie. deposits is masked
Masked index: 28 became amount
Masked index: 30 became loans
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'reserve', 'bank', 'of', 'india', 'has', 'also', 'made', 'a', 'large', 'amount', 'of', 'loans', ',', 'which', 'is', 'a', 'good', 'thing', '.', '[SEP]']
20 ie. bank is masked
27 ie. large is masked
Masked index: 20 became bank
Masked index: 27 became large
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'reserve', 'bank', 'of', 'india', 'has', 'also', 'made', 'a', 'large', 'amount', 'of', 'loans', ',', 'which', 'is', 'a', 'good', 'thing', '.', '[SEP]']
21 ie. of is masked
37 ie. . is masked
Masked index: 21 became of
Masked index: 37 became .
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substant

Masked index: 33 became is
Masked index: 37 became .
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'central', 'bank', 'of', 'india', 'has', 'also', 'provided', 'a', 'large', 'number', 'of', 'loans', ',', 'which', 'is', 'a', 'good', 'thing', '.', '[SEP]']
22 ie. india is masked
29 ie. of is masked
Masked index: 22 became india
Masked index: 29 became of
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'central', 'bank', 'of', 'india', 'has', 'also', 'provided', 'a', 'large', 'number', 'of', 'loans', ',', 'which', 'is', 'a', 'good', 'thing', '.', '[SEP]']
22 ie. india is masked
36 ie. thing is masked
Masked index: 22 became india
Masked index: 36 became example
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country

Masked index: 22 became india
Masked index: 37 became .
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'reserve', 'bank', 'of', 'india', 'is', 'also', 'having', 'a', 'large', 'number', 'of', 'reserves', ',', 'which', 'is', 'a', 'good', 'indicator', '.', '[SEP]']
22 ie. india is masked
36 ie. indicator is masked
Masked index: 22 became india
Masked index: 36 became example
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'reserve', 'bank', 'of', 'india', 'is', 'also', 'having', 'a', 'large', 'number', 'of', 'reserves', ',', 'which', 'is', 'a', 'good', 'example', '.', '[SEP]']
23 ie. is is masked
37 ie. . is masked
Masked index: 23 became is
Masked index: 37 became .
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'co

Masked index: 34 became an
Masked index: 35 became an
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'neighbouring', 'state', 'in', 'victoria', 'is', 'home', 'to', 'a', 'large', 'number', 'of', 'reserves', ',', 'and', 'is', 'an', 'an', 'example', '.', '[SEP]']
22 ie. victoria is masked
28 ie. number is masked
Masked index: 22 became victoria
Masked index: 28 became number
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'neighbouring', 'state', 'in', 'victoria', 'is', 'home', 'to', 'a', 'large', 'number', 'of', 'reserves', ',', 'and', 'is', 'an', 'an', 'example', '.', '[SEP]']
35 ie. an is masked
36 ie. example is masked
Masked index: 35 became important
Masked index: 36 became example
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reser

Masked index: 26 became a
Masked index: 28 became number
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'australian', 'state', 'of', 'victoria', 'is', 'home', 'to', 'a', 'large', 'number', 'of', 'butterflies', ',', 'which', 'are', 'critically', 'important', 'species', '.', '[SEP]']
18 ie. the is masked
21 ie. of is masked
Masked index: 18 became the
Masked index: 21 became of
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'australian', 'state', 'of', 'victoria', 'is', 'home', 'to', 'a', 'large', 'number', 'of', 'butterflies', ',', 'which', 'are', 'critically', 'important', 'species', '.', '[SEP]']
27 ie. large is masked
31 ie. , is masked
Masked index: 27 became large
Masked index: 31 became ,
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantia

...

Masked index: 29 became but
Masked index: 37 became .
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'inhabited', 'by', 'the', 'endemic', 'species', ',', 'but', 'also', 'contains', 'several', 'other', 'endemic', 'plant', 'species', '.', '[SEP]']
20 ie. area is masked
28 ie. , is masked
Masked index: 20 became area
Masked index: 28 became ,
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'inhabited', 'by', 'the', 'endemic', 'species', ',', 'but', 'also', 'contains', 'several', 'other', 'endemic', 'plant', 'species', '.', '[SEP]']
29 ie. but is masked
36 ie. species is masked
Masked index: 29 became but
Masked index: 36 became species
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 

Masked index: 22 became mainly
Masked index: 29 became but
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'inhabited', 'by', 'the', 'endemic', 'species', ',', 'but', 'also', 'also', 'some', 'other', 'endemic', 'plant', 'species', '.', '[SEP]']
24 ie. by is masked
31 ie. also is masked
Masked index: 24 became by
Masked index: 31 became contains
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'inhabited', 'by', 'the', 'endemic', 'species', ',', 'but', 'also', 'contains', 'some', 'other', 'endemic', 'plant', 'species', '.', '[SEP]']
23 ie. inhabited is masked
24 ie. by is masked
Masked index: 23 became composed
Masked index: 24 became for
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very

Masked index: 22 became mainly
Masked index: 26 became animal
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'composed', 'of', 'rare', 'animal', 'species', ',', 'but', 'also', 'includes', 'some', 'rare', 'endemic', 'plant', 'species', '.', '[SEP]']
18 ie. the is masked
32 ie. some is masked
Masked index: 18 became the
Masked index: 32 became some
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'composed', 'of', 'rare', 'animal', 'species', ',', 'but', 'also', 'includes', 'some', 'rare', 'endemic', 'plant', 'species', '.', '[SEP]']
28 ie. , is masked
33 ie. rare is masked
Masked index: 28 became ,
Masked index: 33 became rare
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substan

Masked index: 25 became endemic
Masked index: 37 became .
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'composed', 'of', 'endemic', 'bird', 'species', ',', 'but', 'also', 'contains', 'some', 'rare', 'endemic', 'plant', 'species', '.', '[SEP]']
24 ie. of is masked
32 ie. some is masked
Masked index: 24 became of
Masked index: 32 became some
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'composed', 'of', 'endemic', 'bird', 'species', ',', 'but', 'also', 'contains', 'some', 'rare', 'endemic', 'plant', 'species', '.', '[SEP]']
23 ie. composed is masked
33 ie. rare is masked
Masked index: 23 became composed
Masked index: 33 became rare
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very'

Masked index: 19 became protected
Masked index: 35 became animal
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'composed', 'of', 'endemic', 'plant', 'species', ',', 'but', 'also', 'contains', 'some', 'rare', 'endemic', 'animal', 'species', '.', '[SEP]']
18 ie. the is masked
28 ie. , is masked
Masked index: 18 became the
Masked index: 28 became ,
['[CLS]', 'all', 'of', 'which', 'comprises', 'a', 'very', 'substantial', 'reserve', 'in', 'the', 'country', 'at', 'the', 'present', 'time', '.', '[SEP]', 'the', 'protected', 'area', 'is', 'mainly', 'composed', 'of', 'endemic', 'plant', 'species', ',', 'but', 'also', 'contains', 'some', 'rare', 'endemic', 'animal', 'species', '.', '[SEP]']
31 ie. contains is masked
33 ie. rare is masked
Masked index: 31 became includes
Masked index: 33 became rare
['[CLS]', 'all', 'of', 'which', 'comprises', 'a',